# Challenge - DS - Quilmes 
## Contexto
La empresa __Quilmes__ se encarga de las ventas de bienes de consumo masivos en un segmento B2B y cuenta con diferentes variedades de clientes que se conocen como puntos de ventas, la misma tiene una gran red de vendedores que se encargan de visitar a estos clientes de manera presencial.

El equipo de __Nivel De Servicio__ esta a cargo de todo lo relacionado a la experiencia del cliente en la venta Offline, es decir, a través de una visita del vendedor presencial, dentro de esto mismo puede ver diferentes aspectos como mejorar la calidad y optimizar los tiempos de las visitas de los vendedores, entender que productos son mas recomendable vender en los diferentes clientes, entender que tan eficientes son las visitas que se hacen, entre otras cosas.

El equipo de Nivel De Servicios contacto al de __Ciencia de Datos__ para entender cuál es la mejor manera de optimizar los tiempos de visita en los clientes, para lograr este objetivo se te solicita que generes un producto que permita entender cuál es la recomendación de tiempo de visita para cada uno de los clientes en base a distintas variables comerciales.

## Data 
Se suministra la siguiente información: 

dataset.csv 

- **cliente_id**: Codigo correspondiente a cada clientes 
- **aniomes**: Periodo correspondiente al registro año mes en formato yyyyMM 
- **canal**: Canal al que pertenece el clientes (Autoservicio, Kiosco/Maxiquiosco) 
- **region**: Region de la cual forma parte el cliente 
- **minutos_visitados_mes**: Minutos totales en el mes que se visito el cliente en 
cada periodo 
- **cantidad_heladeras**: Cantidad de heladeras que tenia el cliente en el periodo 
- **venta_total_negocios_mes**: Venta del total de los negocios que tuvo el cliente 
unidad Hl vendidos, esta venta puede no corresponder a la suma de los 
negocios por separados ya que incluye algunos productos que no se 
encuentran en los negocios (1,2,3,4) 
- **cantidad_productos_total_negocios_vendidos_mes**: Cantidad de productos 
distintos vendidos en la venta total de todos los negocios en cada periodo 
- **cantidad_compras_total_negocios_mes**: Cantidad de compras totales que el 
cliente hizo en el periodo 
- **dias_entre_compras_total_negocios_mes**: días pasados entre cada compra 
realizada dentro del periodo 
- **venta_negocio1_mes**: Venta correspondiente al negocio 1 en cada periodo, 
unidad Hl vendidos 
- **venta_negocio2_mes**: Venta correspondiente al negocio 2 en cada periodo, 
unidad Hl vendidos 
- **venta_negocio3_mes**: Venta correspondiente al negocio 3 en cada periodo, 
unidad Hl vendidos 
- **venta_negocio4_mes**: Venta correspondiente al negocio 4 en cada periodo, 
unidad Hl vendidos

## Objetivos
Esta herramienta permite estimar y priorizar el tiempo de visita mensual asignado a cada cliente, en base a su comportamiento reciente.

El objetivo principal es facilitar la toma de decisiones comerciales mediante un ranking regional que indique a qué clientes conviene dedicar más tiempo de visita, optimizando así la eficiencia de los recursos en el territorio.

Variable objetivo (target):

**minutos_visitados_mes**, que representa la duración total (en minutos) que se dedicó a visitar a un cliente en un mes determinado.

A partir de esta variable, se construye un score normalizado para generar rankings comparables entre regiones.

##  Tabla de Contenidos

1. [Importar librerías y raw data](#imports)  
2. [EDA + Preprocessing](#eda-visual)  
3. [Feature Transformation](#features)  
4. [Model Training & Evaluation](#model)
5. [Predicting on New Input Data](#7-predicting-on-new-input-data)

### 1. Importar librerías y raw data

In [ ]:
import sys
sys.path.insert(0, './src/')
import pandas as pd
import importlib
import read_data
importlib.reload(read_data)
from read_data import load_dataset
import clean
importlib.reload(clean)
from clean import cleaning_pipeline


In [25]:
# Load Dataset
df = load_dataset()
df.head()

,cliente_id,aniomes,fecha,canal,region,minutos_visitados_mes,cantidad_heladeras,venta_total_negocios_mes,cantidad_productos_total_negocios_vendidos_mes,cantidad_compras_total_negocios_mes,dias_entre_compras_total_negocios_mes,venta_negocio1_mes,venta_negocio2_mes,venta_negocio3_mes,venta_negocio4_mes
0,3e1f078ba07f177421f0aefa1884347d,202501,2025-01-01,AUTOSERVICIO,LITORAL,31.600000,5.0,5.00946,23.0,45.0,0.577778,2.06946,2.94,0.000,0.00
1,58ae3b6ad98d3e3e4ba2b404c9142e0f,202504,2025-04-01,AUTOSERVICIO,GBA MINORISTAS,214.683333,4.0,6.65658,11.0,19.0,1.526316,0.19158,1.83,4.275,0.36
2,f3a854fb42adecbc313785f1f1629ffe,202407,2024-07-01,KIOSCOS/MAXIKIOSCOS,NEA,17.433333,0.0,0.00000,0.0,0.0,0.000000,0.00000,0.00,0.000,0.00
3,e7d278a2f3c239afeaf9526d7e275401,202412,2024-12-01,KIOSCOS/MAXIKIOSCOS,LITORAL,24.733333,2.0,0.00000,0.0,0.0,0.000000,0.00000,0.00,0.000,0.00
4,3f4bd1f117b18afa131c48de15486652,202411,2024-11-01,KIOSCOS/MAXIKIOSCOS,GBA MINORISTAS,11.166667,2.0,0.00000,0.0,0.0,0.000000,0.00000,0.00,0.000,0.00


### 2. EDA y Preprocessing

En esta sección se ejecuta el pipeline de preprocesamiento definido a partir del análisis exploratorio realizado en la notebook `notebook/EDA.ipynb`.  
El objetivo es limpiar, transformar y preparar los datos para su posterior modelado, asegurando la calidad y consistencia de las variables relevantes para la estimación del tiempo de visita a clientes.

In [26]:
target = 'minutos_visitados_mes'
cols_numericas = df.select_dtypes(include='number').drop(columns=['aniomes']).columns.tolist()
df_clean = cleaning_pipeline(df, target, cols_numericas)
df_clean.head()

,cliente_id,aniomes,fecha,canal,region,minutos_visitados_mes,cantidad_heladeras,venta_total_negocios_mes,cantidad_productos_total_negocios_vendidos_mes,cantidad_compras_total_negocios_mes,dias_entre_compras_total_negocios_mes,venta_negocio1_mes,venta_negocio2_mes,venta_negocio3_mes,venta_negocio4_mes
0,3e1f078ba07f177421f0aefa1884347d,202501,2025-01-01,AUTOSERVICIO,LITORAL,31.600000,5.0,5.00946,23.0,45.0,0.577778,2.06946,2.94,0.000,0.00
1,58ae3b6ad98d3e3e4ba2b404c9142e0f,202504,2025-04-01,AUTOSERVICIO,GBA MINORISTAS,214.683333,4.0,6.65658,11.0,19.0,1.526316,0.19158,1.83,4.275,0.36
2,f3a854fb42adecbc313785f1f1629ffe,202407,2024-07-01,KIOSCOS/MAXIKIOSCOS,NEA,17.433333,0.0,0.00000,0.0,0.0,0.000000,0.00000,0.00,0.000,0.00
3,e7d278a2f3c239afeaf9526d7e275401,202412,2024-12-01,KIOSCOS/MAXIKIOSCOS,LITORAL,24.733333,2.0,0.00000,0.0,0.0,0.000000,0.00000,0.00,0.000,0.00
4,3f4bd1f117b18afa131c48de15486652,202411,2024-11-01,KIOSCOS/MAXIKIOSCOS,GBA MINORISTAS,11.166667,2.0,0.00000,0.0,0.0,0.000000,0.00000,0.00,0.000,0.00


In [27]:
df_clean.describe()

,aniomes,fecha,minutos_visitados_mes,cantidad_heladeras,venta_total_negocios_mes,cantidad_productos_total_negocios_vendidos_mes,cantidad_compras_total_negocios_mes,dias_entre_compras_total_negocios_mes,venta_negocio1_mes,venta_negocio2_mes,venta_negocio3_mes,venta_negocio4_mes
count,196490.000000,196490,196490.000000,196490.000000,196490.000000,196490.000000,196490.000000,196490.000000,196490.000000,196490.000000,196490.000000,196490.000000
mean,202441.928571,2024-10-16 01:42:51.428571392,57.098867,1.000499,2.729932,6.702921,11.669194,3.746393,1.370479,0.853189,0.462124,0.014494
min,202404.000000,2024-04-01 00:00:00,0.000000,0.000000,-4.652640,0.000000,0.000000,0.000000,-3.405600,-8.165000,-10.035000,0.000000
25%,202407.000000,2024-07-01 00:00:00,14.916667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,202410.500000,2024-10-16 12:00:00,30.083333,0.000000,0.210000,4.000000,5.000000,1.142857,0.000000,0.000000,0.000000,0.000000
75%,202502.000000,2025-02-01 00:00:00,62.133333,1.000000,1.450000,9.000000,16.000000,3.500000,0.628380,0.442500,0.000000,0.000000
max,202505.000000,2025-05-01 00:00:00,1708.666667,78.000000,2069.747220,111.000000,591.000000,715.000000,1067.239360,2069.708840,678.780000,405.000000
std,45.574959,NaN,78.170251,1.707791,16.765437,9.174773,19.322290,11.748858,9.931935,9.934206,3.954440,1.237276
